# Событийная аналитика для стартапа, продающего продукты питания. Анализ А/А/В-теста.
***

# Описание проекта

Нужно проанализирвоать поведение пользователей мобильного приложения стартапа, продающего продукты. Исследовать результаты A/A/B-эксперимента замены шрифта в приложении.

#### Описание данных для исследования

Для анализа представылены логи со следующими колонками:
- EventName — название события;
- DeviceIDHash — уникальный идентификатор пользователя;
- EventTimestamp — время события;
- ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

# Оглавление<a class='anchor' id='TOC'></a>

* **[I. Подготовка к анализу](#1)**
* **[2. Исследовательский анализ данных](#2)**
    - [1) Общие сведения о событиях и пользователях](#2_1)
    - [2) Проверим период логирования](#2_2)
* **[3. Анализ поведения пользователей](#3)**
    - [1) Общие сведения о поведении пользователей](#3_1)
    - [2) Воронка событий](#3_2)
* **[4. Анализ A/A/B-теста](#4)**
    - [1) Проверим корректность разбиения групп](#4_1)
    - [2) Проведем статистические тесты для контрольных групп](#4_2)
    - [3) Проведем статистические тесты для тестовой группы](#4_3)
* **[Выводы](#conclusions)**
<br></br>

# I. Подготовка к анализу<a class='anchor' id='1'></a>

## 1) Импортируем библиотеки, объявим класс и несколько полезных функций для анализа данных<a class="anchor" id="1_1"></a>

Импортируем библиотеки:

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
from scipy.stats import shapiro
from statsmodels.stats.proportion import proportions_ztest

# Определяем способ вывода графиков plotly
pio.renderers.default = 'iframe'

# Отключаем предупреждения
warnings.filterwarnings('ignore')


In [12]:
import pandas as pd
from solver.data_reader import DataReader, get_profiles, simple_grouper

[В оглавление](#TOC)

## 2) Прочитаем данные<a class="anchor" id="1_2"></a>

### - инициализируем класс

In [13]:
reader = DataReader('logs_exp.csv')

### - выведем первые пять первых и последних строк таблицы , общую информацию, статиcтику

In [14]:
reader.basic_info_printer(reader.origin_data)

'Пять первых и последних строк'

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
...,...,...,...,...
244121,MainScreenAppear,4599628364049201812,1565212345,247
244122,MainScreenAppear,5849806612437486590,1565212439,246
244123,MainScreenAppear,5746969938801999050,1565212483,246
244124,MainScreenAppear,5746969938801999050,1565212498,246


'Общая информация о датасете'

,Column,Dtype,Non-Null Count,% of nulls
0,EventName,object,244126,0.0
1,DeviceIDHash,int64,244126,0.0
2,EventTimestamp,int64,244126,0.0
3,ExpId,int64,244126,0.0


'Описательная статистика'

,count,unique,top,freq,% of tops freq
EventName,244126,5,MainScreenAppear,119205,48.83


&#9889; **Выводы**

1. В представленных данных 244126 наблюдений, без явных пропусков, без явных проблем с типом данных.
2. Судя по формату в столбце "EventTimestamp" - время события записано в формате [Unix-времени](https://ru.wikipedia.org/wiki/Unix-%D0%B2%D1%80%D0%B5%D0%BC%D1%8F), то есть в секундах (для минут указывают дополнительнео minutes, для наносекунд ns).
3. Всего представленны данные о пяти различных события, самое частое из которых - загрузка заглавной страницы (119 205 из 244 126 наблюдений, 49%).
4. Потребуется пересчитать дату - получить столбец с датой и временем в формате ISO и столбец только с датой.
5. Кроме того, переименуем для ясности колонки, события, группы.

### - пересчитаем дату события, создадим столбцы с полным временем и, отдельно, датой, переименуем колонки, события и группы

In [15]:
logs = reader.data_cleaner()
logs

,event,id,origin_timestamp,group,timestamp,date
0,main screen,4575588528974610257,1564029816,A_one,2019-07-25 04:43:36,2019-07-25
1,main screen,7416695313311560658,1564053102,A_one,2019-07-25 11:11:42,2019-07-25
2,payment,3518123091307005509,1564054127,B_test,2019-07-25 11:28:47,2019-07-25
3,cart,3518123091307005509,1564054127,B_test,2019-07-25 11:28:47,2019-07-25
4,payment,6217807653094995999,1564055322,B_test,2019-07-25 11:48:42,2019-07-25
...,...,...,...,...,...,...
244121,main screen,4599628364049201812,1565212345,A_two,2019-08-07 21:12:25,2019-08-07
244122,main screen,5849806612437486590,1565212439,A_one,2019-08-07 21:13:59,2019-08-07
244123,main screen,5746969938801999050,1565212483,A_one,2019-08-07 21:14:43,2019-08-07
244124,main screen,5746969938801999050,1565212498,A_one,2019-08-07 21:14:58,2019-08-07


Получили таблицу с шестью столбцами, 244 126 строками. Наименования столбцов:
- events - события,
- user_id - хэш пользователя,
- origin_timestamp - оригинальная информация о времени события,
- group - сведения о группе,
- timestamp - пересчитанная дата и время,
- date - дата события.

### - выведем первые пять первых и последних строк полученной таблицы, общую информацию, статиcтику результата

In [16]:
reader.basic_info_printer(logs)

'Пять первых и последних строк'

,event,id,origin_timestamp,group,timestamp,date
0,main screen,4575588528974610257,1564029816,A_one,2019-07-25 04:43:36,2019-07-25
1,main screen,7416695313311560658,1564053102,A_one,2019-07-25 11:11:42,2019-07-25
2,payment,3518123091307005509,1564054127,B_test,2019-07-25 11:28:47,2019-07-25
3,cart,3518123091307005509,1564054127,B_test,2019-07-25 11:28:47,2019-07-25
4,payment,6217807653094995999,1564055322,B_test,2019-07-25 11:48:42,2019-07-25
...,...,...,...,...,...,...
244121,main screen,4599628364049201812,1565212345,A_two,2019-08-07 21:12:25,2019-08-07
244122,main screen,5849806612437486590,1565212439,A_one,2019-08-07 21:13:59,2019-08-07
244123,main screen,5746969938801999050,1565212483,A_one,2019-08-07 21:14:43,2019-08-07
244124,main screen,5746969938801999050,1565212498,A_one,2019-08-07 21:14:58,2019-08-07


'Общая информация о датасете'

,Column,Dtype,Non-Null Count,% of nulls
0,event,object,244126,0.0
1,id,int64,244126,0.0
2,origin_timestamp,int64,244126,0.0
3,group,object,244126,0.0
4,timestamp,datetime64[ns],244126,0.0
5,date,object,244126,0.0


'Описательная статистика'

,count,unique,top,freq,% of tops freq,first,last
timestamp,244126,176654,2019-08-01 14:40:35,9,0.00,2019-07-25 04:43:36,2019-08-07 21:15:17
date,244126,14,2019-08-01,36229,14.84,---,---
group,244126,3,B_test,85747,35.12,---,---
event,244126,5,main screen,119205,48.83,---,---


### - посчитаем количество дубликатов

In [17]:
logs.duplicated().sum()

413

### - исключим дубликаты

Поскольку выявленны именно полные дубликаты - принимая во внимание, что в том числе это дубликаты по полной дате - в отстутвии возможности уточнить у поставщика данных природу возникновения дубликатов, учитывая что 413 наблдений составляют менее 1% от наблюдений: принято решение исключить их из исследования на стадии подготовки, предварительно сохранив копию.

In [18]:
duplicates = logs[logs.duplicated()].copy()

In [19]:
logs.drop_duplicates(inplace=True)
logs

,event,id,origin_timestamp,group,timestamp,date
0,main screen,4575588528974610257,1564029816,A_one,2019-07-25 04:43:36,2019-07-25
1,main screen,7416695313311560658,1564053102,A_one,2019-07-25 11:11:42,2019-07-25
2,payment,3518123091307005509,1564054127,B_test,2019-07-25 11:28:47,2019-07-25
3,cart,3518123091307005509,1564054127,B_test,2019-07-25 11:28:47,2019-07-25
4,payment,6217807653094995999,1564055322,B_test,2019-07-25 11:48:42,2019-07-25
...,...,...,...,...,...,...
244121,main screen,4599628364049201812,1565212345,A_two,2019-08-07 21:12:25,2019-08-07
244122,main screen,5849806612437486590,1565212439,A_one,2019-08-07 21:13:59,2019-08-07
244123,main screen,5746969938801999050,1565212483,A_one,2019-08-07 21:14:43,2019-08-07
244124,main screen,5746969938801999050,1565212498,A_one,2019-08-07 21:14:58,2019-08-07


### - проверим последовательно ли идут события

Убедимся в том, что события представлены последовательно во времени, поскольку это критично для целей анализа -в том числе с целью определения последовательности взаимодействия с приложением:

In [20]:
logs['timestamp'].is_monotonic

True

&#9889; **Выводы**

1. В представленных данных наблюдения за две недели: от 25 июля 2019 года до 07.08.2019 года. Данные идут последовательно.
2. Больше всего наблюдений от 01 августе 2019 года (36 299 из 244 126, 15%).
3. Больше всего наблюдений о тестовой группе В - 35% (85 747). Требуется дополнительно проверить разбиение групп.
4. В данных обнаружено 413 полных дубликатов, принято решение об исключении дубликатов на стадии подготовки: получен датасет в 243 713 строк и 6 колонок.

[В оглавление](#TOC)

# II. Исследовательский анализ данных<a class='anchor' id='2'></a>

## 1) Общие сведения о событиях и пользователях <a class="anchor" id="2_1"></a>

### - cколько всего событий и пользователей в логе

In [21]:
# Группируем данные по событиям, считаем уникальных пользователей
events_and_users = logs.groupby('event', as_index=False).agg({'id': 'nunique'})

# Считаем общее число уникальных пользователей
events_and_users['total_users'] = logs['id'].nunique()

# Считаем долю пользователей для каждого события
events_and_users['percent'] = round(events_and_users['id'] * 100 / 
                                    events_and_users['total_users'], 2)

# Переименовываем и сортируем результат
events_and_users.rename(columns={'id': 'unique_users'}, inplace=True)
events_and_users.sort_values(by='percent', ascending=False)

,event,unique_users,total_users,percent
1,main screen,7439,7551,98.52
2,offer screen,4613,7551,61.09
0,cart,3749,7551,49.65
3,payment,3547,7551,46.97
4,tutorial,847,7551,11.22


&#9889; **Выводы**

1. В представленном датасете 243 713 наблюдений о 7 551 уникальном пользователе.
2. Всего в датасете представлено пять событий:
    - main screen (посетило 7 439 уникальных пользователей из 7 551, 98.47%)
    - offer screen (посетило 4613 уникальных пользователей из 7 551, 60.96%)
    - cart (посетило 3749 уникальных пользователей из 7 551, 49.56%)
    - payment (посетило 3547 уникальных пользователей из 7 551, 46.97%)
    - tutorial (посетило 847 уникальных пользователей из 7 551, 11.15%)


3. В целом, почти половина из тех, кто пришел - что-то купили.

### - cколько в среднем событий приходится на пользователя

Чтобы посчитать сколько в среднем событый приходится на пользователя воспользуемся группировкой:

In [22]:
# Посчитам сколько из каждого вида событый у каждого пользоватлей
count_events = logs.groupby(['event', 'id'],
                            as_index=False).agg({'date': 'count'})

# Сгруппируем результат по событию и посчитаем среднее
mean_events = count_events.groupby(
    'event', as_index=False).agg({'date': 'mean'})

# Переименуем колонку со средним
mean_events.rename(columns={'date': 'mean_per_user'}, inplace=True)

# Отсортируем результат
mean_events.sort_values(by='mean_per_user', ascending=False, inplace=True)

# Добавим среднее на пользователя по событяим в совокупности
result_row = pd.Series(['All events',
                        len(logs) / logs['id'].nunique()],
                       index=mean_events.columns)
mean_events = mean_events.append(result_row, ignore_index=True)

mean_events

,event,mean_per_user
0,main screen,16.010351
1,cart,11.381168
2,offer screen,10.146976
3,payment,9.618833
4,tutorial,1.201889
5,All events,32.275593


&#9889; **Выводы**

1. В среднем, на пользователя приходится 32.27 событий.
2. Из пяти событий в представленных данных на пользователя, в среднем, приходится:
    - 16 посещений главного экрана,
    - 11.38 посещений корзины,
    - 10.14 посещенйй страницы с предложениями,
    - 9.6 платежей,
    - 1.2 обучение
3. Как следствие, средний пользователь один раз проходит обучение, но зато потом почти десять раз купит что-то.

[В оглавление](#TOC)

## 2) Сведения о периоде логирования <a class="anchor" id="2_2"></a>

### - период логирования, максимальная и минимальные даты записи

In [23]:
reader.get_describe(logs)

,count,unique,top,freq,% of tops freq,first,last
timestamp,243713,176654,2019-08-01 14:40:35,9,0.00,2019-07-25 04:43:36,2019-08-07 21:15:17
date,243713,14,2019-08-01,36141,14.83,---,---
group,243713,3,B_test,85582,35.12,---,---
event,243713,5,main screen,119101,48.87,---,---


Период логирования - с 25.07.2019 до 07.08.2019 года, две недели.


### - построим гистограмму по дате и времени

In [ ]:
def histogram(data, bins='rice', labels=None, stats=False):
    """Строим гистограмму."""

    # если не переданы подписи
    if not labels:
        labels = {'xlabel': '',
                  'title': ''}

    # Задаем стиль
    sns.set_style('whitegrid')

    plt.figure(figsize=(15, 10))

    # Строим график
    plt.hist(data, bins=bins, color='#575fcf')

    # Добавим легкие вертикальные линии
    plt.grid(axis='x', alpha=0.0)
    plt.grid(axis='y', alpha=0.15)

    # Добавим среднее, медиану, 1-й и 99-й перцентили
    if stats:
        plt.axvline(data.median(), color='#ffd32a',
                    label='медиана', linestyle='--')
        plt.axvline(data.mean(), color='#ff3f34',
                    label='среднее', linestyle='--')
        plt.axvline(data.quantile(0.1), color='#1e272e',
                    label='1%', linestyle='--')
        plt.axvline(data.quantile(0.99), color='#1e272e',
                    label='99%', linestyle='--')
        plt.legend()

    # Подпишем оси и график
    plt.xlabel(labels['xlabel'], labelpad=30)
    plt.ylabel('Число наблюдений')
    plt.title(labels['title'], loc='left', pad=30,
              fontsize=12, fontweight='bold')


Подготовим подписи для графика:

In [ ]:
labels = {'xlabel': 'Дата наблюдений',
          'title': 'Наблюдения из логов по дате и времени'}

Построим гистограмму:

In [ ]:
histogram(logs.timestamp, labels=labels)

Исходя из гистограммы можно сделать вывод о том, что данных за июль, фактически, нет - наблюдений сравнительно мало. Кроме того, у нас мультимодальное распределние - принимая во внимание наличие семи выраженных пиков с провалами, с некоторой осторожностью, можно предположить, что во временном ряде наблюдаются дневные циклы.

### - оценим полноту данных

Сгруппируем данные по дате и посчитаем количество наблюдений по каждому дню:

In [ ]:
grouped_by_date = simple_grouper(logs, 'date')
grouped_by_date

Табличные данные подтверждают относительную малочисленность данных за июль замеченные на гистограмме: за период от 25.07.2019 года до 31.07.2019 года зафиксировано чуть более 1% всех наблюдений о пользователях. Посчитаем сколько наблюдений попало в период от 25 до 31 июля:

In [ ]:
grouped_by_date.query('percent < 1').sum()

Всего 2826 из 243 713 наблюдений, 1.15%, попали в период от 25 до 31 июля. Для целей анализа, принимая во внимание малочисленность наблюдений за этот период, данные за указанный период можно исключить.

### - исключим неполные логи

Сохраним также логи за июль: это повод обратиться к поставщику данных, возможно, стоит проверить правильно ли ведутся ли логи.

In [ ]:
july_logs = logs.query('timestamp < "2019-08-01"').copy()
july_logs

Сделаем срез за август, выведем общую информацию:

In [ ]:
logs = logs.query('timestamp >= "2019-08-01"').reset_index(drop=True)
reader.basic_info_printer(logs)

&#9889; **Выводы**
1. После исключения данных за июль получили таблицу с 240 887 строками и 6 колонками, без пропусков и проблем с данными.
2. Период наблюдений от 01.08.2019 года до 07.08.2019 года.

### - проверим группы пользователей

Соберем профили пользователей:

In [ ]:
def get_profiles(data):
    """Cоздаем пользовательские профили."""

    # Находим параметры первых посещений
    profiles = (data
                .sort_values(by=['id', 'date'])
                .groupby('id').agg({'timestamp': 'first',
                                    'date': 'first',
                                    'group': 'first'})
                .rename(columns={'date': 'first_ts'})
                .reset_index())

    # Для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['timestamp'].dt.date
    profiles['month'] = profiles['timestamp'].astype('datetime64[M]')

    # Добавим в профиль сведения о последнем событии для каждого пользователя
    last_event = (data.groupby(['id', 'event', 'date'])
                  .last()
                  .reset_index()
                  .drop_duplicates(subset=['id'], keep='last')
                  [['id', 'event']])
    last_event.columns = ['id', 'last_event']
    profiles = profiles.merge(last_event)

    # Отбираем заплативших пользователей
    orders = data.query('event == "payment"')

    # Добавляем признак платящих пользователей
    profiles['payer'] = profiles['id'].isin(orders['id'].unique())

    return profiles


In [ ]:
profiles = get_profiles(logs)
profiles

После исключения наблюдений за июль в таблице остались сведения о 7534 из 7551 уникальных пользователей (99,7%). Сгруппируем данные по группам, проверим сколько пользователей попало в каждую: 

In [ ]:
simple_grouper(profiles, 'group')

В датасете представлены три примерно равные группы, с разницей в численности каждой из групп менее 1%.

&#9889; **Выводы**

[В оглавление](#TOC)

# III. Анализ поведения пользователей<a class='anchor' id='3'></a>

## 1) Общие сведения о поведении пользователей <a class="anchor" id="3_1"></a>

### - типы событий в логах, частота событый

Ранее установлено, с какими событиями сталкиваются пользователи. В связи с невозможностью уточнить данные, исходим из того, что: 
- main screen - главная страница магазина,
- offer screen - страница товара,
- cart - корзина,
- payment - оплата товара,
- tutorial - обучение.

Выведедем на экран шаги ранжированные по количеству неуникальных пользователей и их доле:

In [ ]:
events_with_all_users = simple_grouper(logs, 'event', agg_dict={'id': 'count'})
events_with_all_users

### - число и доля пользователей по каждому событию

Посчитаем число и долю пользователей на сокращенных данных:

In [ ]:
# Сохраним в переменную число уникальных пользователей
total_users = logs['id'].nunique()

# Сгруппируем таблицу по событиям, посчитаем число уникальных пользователей на каждом
events_and_users = logs.groupby('event', as_index=False).agg({'id': 'nunique'})

# Посчитаем конверсию
events_and_users['percent'] = round(events_and_users['id'] * 100 / 
                                       total_users, 2)

# Переименуем колонку с id для ясности
events_and_users.rename(columns={'id': 'unique_users'}, inplace=True)
events_and_users.sort_values(by='percent', ascending=False, inplace=True)
events_and_users

До олаты добираются почти 47% процентов пользователей. На первый взгляд (и представляется, что это не далеко от истины), что количество и доля пользователей хороший маркер для выявления последовательности событий, но, принимая во внимание, что шаг с обучением проходят всего 11.15% уникальных пользователей, а также неполную ясность назначения "offer screen"  - паттерн требует некоторого уточнения.

### - последовательность событий в логах

Исходя из того, что мы выявили монотонное увеличение в дате и времени - все события последовательны. Следовательно, если предположить, что события также и взаимосвязаны, одно не может быть осуществлено без предыдущего, то можем исходить из того, что как бы пользователи не действовали - в целом, шаги будут пройдены последовательно. Следовательно, для выявления временной поседовательности событий, отсеим дубликаты в колонке с событиями, сохранив только первое появление каждого значения и дополним результат числом и долей пользователей по каждому событию:

In [ ]:
logs.drop_duplicates('event')[['event']].merge(events_and_users, on='event')

&#9889; **Выводы:**

Опираясь на число и долю пользователей и повременное появление событий в таблицы - применим наивный подход для выявления последовательности событий. С некоторой осторожностью, можно утверждать, что посетители интернет-магазина до оплаты товара проходят такие шаги:
1. Попадают на экран с обучением, 
2. Заходят на главную страницу магазина.
3. Переходят на страницу товара.
4. Добавляют товар в корзину.
5. Переходят в корзину, оформляют заказ.
6. Оплачивают заказ.

Учитывая, что обучение проходят всего 11.15% уникальных пользователей и, разумно предположить, обучение не загружается каждый раз, этот шаг явялется необязательной частью взаимодействия с магазином.

[В оглавление](#TOC)

## 2) Воронка событий <a class="anchor" id="3_2"></a>

### - подготовим данные

Исключим из числа событий обучение - tutorial. Для воронки со всеми уникальными пользователями возьмем ранее подготовленную аггрегированную таблицу events_and_users.

In [ ]:
event_without_tutorial = events_and_users.query('event != "tutorial"').copy()
event_without_tutorial

Кроме того, нас интересуют как проходили воронку пользователи из разных групп. Соберем данные вместе:

In [ ]:
# Зафикисируем порядок и исключим обучение
order = logs.drop_duplicates('event')[['event']].merge(events_and_users, on='event')
order = order.query('event != "tutorial"').reset_index()

# Сгруппируем логи по событиям и группам, посчитаем уникальных пользователей
groups = logs.groupby(['event', 'group']).agg({'id': 'nunique'}).reset_index()

# Объединим две таблицы и выведем результат
groups = groups.merge(order).sort_values(by='index')
groups

Получили таблицу где к каждому событию привязано число пользователей из каждой группы, учтена последовательность событий. 

### - визуализируем продуктовую воронку

Подготовим функцию для визуализации

In [ ]:
def funnel_plot(data, groups_labels=None,
                x='id', y='event',
                title='', *args, **kwargs):
    fig = go.Figure()
    
    # Если переданы маркеры групп
    if groups_labels:
        for group in groups_labels:
            sample = data.query(f'group == "{group}"')
            fig.add_trace(go.Funnel(
                name=group,
                y=sample[y],
                x=sample[x], *args, **kwargs))

    else: # Строим по всем данным
        fig.add_trace(go.Funnel(name='Все пользователи',
                                y=data[y],
                                x=data[x], *args, **kwargs))

    # Изменим размер, фон, добавим заголовок
    fig.update_layout(autosize=False,
                      width=700,
                      height=500,
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)',
                      title=title)
    fig.show()

Сначала по всем пользователям:

In [ ]:
event_without_tutorial

In [ ]:
funnel_plot(data=event_without_tutorial, 
            groups_labels=[], 
            title='Воронка событий по всем пользователям',
            x='unique_users')

Теперь примем во внимание группы пользователей:

In [ ]:
funnel_plot(data=groups, 
            groups_labels=['A_one', 'A_two', 'B_test'], 
            title='Воронка событий по группам')

### - выявим шаг с наибольшей потерей, посчитаем долю пользователей прошедших от первого события до оплаты

In [ ]:
event_without_tutorial

Посчитаем долю пользователей от шага к шагу:

In [ ]:
event_without_tutorial['diff'] = abs(event_without_tutorial['unique_users'].diff())
event_without_tutorial['% loss next step'] = abs(1 + round(event_without_tutorial['unique_users']
                                                           .pct_change(), 2) * 100)
event_without_tutorial.fillna('---', inplace=True)
event_without_tutorial

Наибольшая потеря происходит при переходе на экран с предложениями товаров - 2 826 (37%) из 7 419 уникальных пользователей так и не перешли с главной страницы. Используем профили пользователей, чтобы проверить связано ли это с конкретным днем:

In [ ]:
# Отберем неплательщиков и платящих, сгруппируем по дате
non_payers = profiles.query('payer == False').groupby('dt').agg({'id': 'count'}).reset_index()
payers = profiles.query('payer == True').groupby('dt').agg({'id': 'count'}).reset_index()

# Объединим полученные таблицы и переименуем столбцы
payer_by_day = non_payers.merge(payers, on='dt')
payer_by_day.columns = ['date', 'non_payers', 'payers']

# Посчитаем абсолютную и относительную разницу
payer_by_day['difference'] = payer_by_day.non_payers - payer_by_day.payers
payer_by_day['% of diff'] = payer_by_day.non_payers / payer_by_day.payers

# Добавим сведения о дне неделе
payer_by_day['day_of_week'] = payer_by_day['date'].astype('datetime64[ns]').dt.day_name()
payer_by_day

Визуализируем отношение неплатящих к платящим по дням:

In [ ]:
# Установим стиль whitegrid из seaborn
sns.set_style('whitegrid')

# Зададим размер
plt.figure(figsize=(10, 7))

#  Построим график
ax = plt.subplot(1, 1, 1)
payer_by_day[['date', '% of diff']].plot(x='date', y=['% of diff'], ax=ax)

# добавим легкие горизонтальные линии
plt.grid(axis='y', alpha=0.15)
plt.grid(axis='x', alpha=0)

# спрячем лишние границы
sns.despine(left=True, bottom=True)

# Подпишем график, оси
plt.xlabel('Дата', labelpad=15)
plt.ylabel('Отношение')
plt.title('Отношение неплатящих к платящим пользователям',
          loc='left', fontsize=13, fontweight='bold', pad=30)
plt.show();

Любопытно - с некоторой осторожностью можно зафиксировать снижение числа платящих, нарастание числа неплатящих. В отстутвие дополнительных данных, принимая во внимание короткий период - нет достаточных оснований для резюмирущих выводов, однако локально стоит обратить внимание на отношение не платящих к платящим.

&#9889; **Выводы**

1. Из 7 419 уникальных пользователей, 2 826 так и не перешли на экран с предложениями товара.
2. Только 46.97% (3 539) прошли от первого до последнего этапа.
3. Из 7 419 уникальных пользователей 2 826 (37%) так и не перешли с главной страницы - это шаг с наибольшей потерей. Кроме того, при переходе с экрана предложений 859 (18%) из 4593 перешедших на экран предложений пользователей не перешли к оформлению заказа. Только 195 (4%) из 3734 так и не оплатили товар. 
4. Следовательно, половина пользователей не доходят до корзины - установить причину исходя из данных не представляется возможным, требуются сведения об устройствах, каналах привлечения, способах оплаты и так далее.
5. Отмечаем в течение недели рост отношения в пользу неплатящих пользователей - в отсутвие данных без конкретных выводов: это повод для более широкого анализа на большем объеме данных.

[В оглавление](#TOC)

# IV. Анализ A/A/B-теста<a class='anchor' id='4'></a>

### - подготовимся к анализу

Создадим класс для статистических тестов, подготовим несколько полезных функций:

In [ ]:
class StatTest:
    """Класс для проведения статистических тестов и сборов результатов."""

    def __init__(self, data, total_ratio=True):
        self.data = data.copy()
        self.events = self.data.query('event != "tutorial"')['event'].unique()
        self.shapiro_flag = False
        self.result = self.df_constructor()
        self.total_ratio = total_ratio
        self.groups_total = (self.data.groupby('group')
                                      .agg({'id': 'nunique'})
                                      .reset_index())
        self.agg_groups = self.group_constructor()
        self.united_group = (self.agg_groups.query('group in ["A_one", "A_two"]')
                                            .groupby('event')
                                            .sum().reset_index())
        
    def df_constructor(self):
        df = pd.DataFrame(columns=['Выборка',
                                   'Нулевая гипотеза (Н0)',
                                   'Альтернативная гипотеза (Н1)',
                                   'alpha', 'p-value',
                                   'p-value < alpha',
                                   'Н0/Н1'])
        return df
    
    def group_constructor(self):
        """Собираем аггрегированные данные по группам."""
        
        # Зафикисируем порядок и исключим обучение
        order = self.data.drop_duplicates('event')[['event']]
        order = order.query('event != "tutorial"').reset_index()

        # Сгруппируем логи по событиям и группам, посчитаем уникальных пользователей
        groups = self.data.groupby(['event', 'group']).agg({'id': 'nunique'}).reset_index()

        # Объединим две таблицы, оставим тольк нужные стобцы
        groups = (groups.merge(order)
                        .sort_values(by='index')[['event', 'group', 'id']]
                        .reset_index(drop=True))

        for group_marker in groups.group.unique():
            row = groups.group == f'{group_marker}'
            
            # Считаем число пользователей на предыдущем шаге           
            groups.loc[row, 'prev'] = (groups.query(f'group == "{group_marker}"').id +
                                       abs(groups.query(f'group == "{group_marker}"').id.diff()))

        for i in groups.group.unique():
            sum_of = int(self.groups_total.query(f'group == "{i}"')['id'])
            groups.loc[groups.group == f'{i}', 'total'] = sum_of
            
        # Исправляем пропук на первом шаге
        groups['prev'].fillna(groups['total'], inplace=True)
        
        return groups
        
    def groups_picker(self, groups, event, united):
        """Выбираем группы из данных."""
        
        if united:
            group_a = self.united_group.query(f'event == "{event}"')
        else:
            group_a = self.agg_groups.query(f'group in {groups[0]} and '
                                            f'event == "{event}"')
        group_b = self.agg_groups.query(f'group in {groups[1]} and '
                                        f'event == "{event}"')

        return (group_a, group_b)

    def shapiro_test(self):
        """Проверка данных на нормальность. Критерий Шапиро-Уилка."""

        # Сгруппируем данные по дате, посчитам число посещений в день
        # оставим для теста только хэши пользователя
        sample = (self.data.groupby('date')
                  .agg({'id': 'count'})
                  .reset_index()['id'])

        p_value = shapiro(sample)
        return p_value[1]

    def z_test(self, samples):
        """Проводим двухвыборочный Z-критерий."""

        group_a = samples[0]
        group_b = samples[1]

        # Фиксируем число попыток
        count = np.array([group_a['id'], group_b['id']])
        
        # Пропорции либо от общего, либо от предыдущего шага
        if self.total_ratio:
            nobs = np.array([group_a['total'], group_b['total']])
        else:
            nobs = np.array([group_a['prev'], group_b['prev']])
            
        # Считаем статистику      
        _, p_value = proportions_ztest(count, nobs)
        return p_value[0]

    def test_designer(self, params):
        """Проводим статистические тесты, собираем результаты."""

        # Получаем данные для теста
        sample_name = params['sample_name']
        alpha = params['alpha']
        zero_hypothesis = params['zero_hypothesis']
        alt_hypothesis = params['alt_hypothesis']

        # Проевряем какой тест проводим
        if params['test'] == 'shapiro':
            # Флаг, чтобы исключить из фиальной таблицы
            self.shapiro_flag = True
            p_value = self.shapiro_test()
        else:
            if self.shapiro_flag:  # Если флаг установлен
                self.result = self.df_constructor() # Очищаем таблицу
                self.shapiro_flag = False
            p_value = self.z_test(params['samples'])

        pvalue_alpha_comparsion = p_value < alpha
        hypo_check = 'Н1' if pvalue_alpha_comparsion else 'Н0'
        row = pd.Series([sample_name, zero_hypothesis,
                        alt_hypothesis, alpha, p_value,
                        pvalue_alpha_comparsion, hypo_check],
                        index=self.result.columns)

        self.result = self.result.append(row, ignore_index=True)

    def test_routine(self, groups_labels, sample_name, united=False):
        """Проводим множественные тесты."""
        for event in list(self.agg_groups.event.unique()):
            # Делаем выборки
            groups_to_test = self.groups_picker(groups_labels, event, united=united)
            params_of_test['samples'] = [*groups_to_test]
            # Передаем имя
            params_of_test['sample_name'] = f'{sample_name}: {event}'
            tests.test_designer(params_of_test)


def report_styler(report):
    """Выводим таблицу с заданным форматом."""
    display(report.style
                  .set_properties(**{'text-align': 'left'})
                  .set_table_styles([{'selector': 'th',
                                      'props': [('text-align', 'left')]}])
                  .format({'alpha': "{:.2f}", 'p-value < alpha': bool}))


Инициализируем класс:

In [ ]:
tests = StatTest(logs)

## 1) Проверим корректность разбиения групп<a class="anchor" id="4_1"></a>

### - проверим чиcло пользователей в каждой из групп

Используем ранее полученные профили пользователей:

In [ ]:
profiles

Посчитаем пользователей по группам:

In [ ]:
simple_grouper(profiles, 'group')

Все три группы примерно одного размера, разница между группами не превышает 1%.

### - проверим пересечения между группами

In [ ]:
def get_intersections(data, groups, test_group=None):
    """Проверяем, что пользователи из групп изолированы."""

    # Будем собирать данные о пересечениях
    result = {}

    # Отбираем группы из данных
    group_a = data.query(f'group == "{groups[0]}"')['id']
    group_b = data.query(f'group == "{groups[1]}"')['id']

    # Получаем пользователей, попавших в обе группы
    group_intersections = list(np.intersect1d(group_a,
                                              group_b))

    # Если группы не изолированы - печатаем сообщение
    if group_intersections:
        print('Группы не изолированы.')
        result['Персечение групп'] = group_intersections

    # Если передан маркер для тестовой группы и нет пересечений
    if test_group:
        test_group = data.query(f'group == "{test_group[0]}"')['id']
        control_group = pd.concat([group_a, group_b])
        control_groups_intersections = list(np.intersect1d(control_group,
                                                           test_group))

        # Если тестовая группа не изолирована от контрольных - сообщение
        if control_groups_intersections:
            print('Тестовая группа не изолирована от контрольных.')
            result['Персечение с тестовой'] = control_groups_intersections
            return result

    # Если группы изолированы - печатаем сообщение
    if not result:
        print('Пересечений нет, группы изолированы.')

    return result


In [ ]:
intersection = get_intersections(logs, groups=['A_one', 'A_two'], test_group=['B_test'])

&#9889; **Выводы**

1. Всего в таблице три группы - две контрольных, одна тестовая: 
    - 2537 (33.67% от общего числа уникальных пользователей),
    - 2513 (33.36% от общего числа уникальных пользователей),
    - 2484 (32.97% от общего числа уникальных пользователей).
2. Все три группы примерно одного размера, разница между группами не превышает 1%.
3. Пересечений между группами нет.

[В оглавление](#TOC)

## 2) Проведем статистические тесты для контрольных групп<a class="anchor" id="4_2"></a>

### - убедимся, что полученная выборка распределена нормально

Проверим гипотезу о том, что данные получены из генеральной совокупности с нормальным распределением, для чего воспользуемся критерием Шапиро-Уилка.

1. Нулевую гипотезу, которую проверяет тест, сформулируем так: "Выборка  нормально распределена",
2. Альтернативную гипотезу, соответственно, сформулируем так: "Распределение выборки не нормально",
3. Примем уровень значимости в 5%.

Подготовим параметры для теста:

In [ ]:
params_of_test = {'sample_name': 'Полная выборка',
                  'alpha': 0.05,
                  'zero_hypothesis': 'Выборка нормально распределена',
                  'alt_hypothesis': 'Распределение выборки не нормально',
                  'test': 'shapiro'}

Проведем тест:

In [ ]:
tests.test_designer(params_of_test)
report_styler(tests.result)

&#9889; **Выводы**

В отношении распределения выборки, исходя из представленных данных, на уровне значимости 5% - нет оснований отвергнуть нулевую гипотезу в пользу альтернативы: выборка нормально распределена.

[В оглавление](#TOC)

### - проведем статистические тесты в отношении контрольных групп

Исходя из результов применения к выборке критерия Шапиро-Уилка на уровне значимости 5% - выборка нормально распределена. Поскольку, принимая во внимаение задачу, нужно проверить повлиял ли новый шрифт на поведение пользователей (безотносительно того как именно повлиял) - нас интересует изменилась ли пропорция пользователей от типичных события к событию в тестовой группе. Учитывая, что получена относительно большая выборка, мы можем применить Z-тест для проверки гипотезы о равенстве долей в группах.

Для начала применим Z-тест к контрольным группам, чтобы убедиться в корректности разделения групп.

Проверим гипотезу о том, что доли пользователей в двух контрольных группах одинаковы на кждом из шагов.

1. Нулевую гипотезу, которую проверяет тест, сформулируем так: "Между долями пользователей в контрольных группах нет статистически значимой разницы",
2. Альтернативную гипотезу, соответственно, сформулируем так: "Между долями пользователей в контрольных группах есть статистически значимая разница",
3. Примем уровень значимости в 5%.

Подготовим параметры для теста:

In [ ]:
sample_name = 'Контрольные группы'
params_of_test = {'sample_name': sample_name,
                  'alpha': 0.05,
                  'zero_hypothesis': 'Между долями пользователей нет различий',
                  'alt_hypothesis': 'Между долями пользователей есть различия',
                  'test': 'z_test'}

Проведем тест и выведем результат:

In [ ]:
tests.test_routine(groups_labels=[['A_one'], ['A_two']], sample_name=sample_name)

Выведем результат:

In [ ]:
report_styler(tests.result)

&#9889; **Выводы**

1. В отношении различий долей между двумя контрольными группами, исходя из представленных данных, на уровне значимости 5% - нет оснований отвергнуть нулевую гипотезу в пользу альтернативы: между долями пользователей в контрольных группах нет статистически значимой разницы.
2. Принимая во внимание, что разница в количестве пользователей между контрольными группами не превышает 1%, между группами нет пересечений, учитывая результаты статистического теста - с некоторой осторожностью, можно утверждать, что группы разеделены корректно.

[В оглавление](#TOC)

## 3) Проведем статистические тесты для тестовой группы<a class="anchor" id="4_3"></a>

### - проведем статистические тесты в отношении контрольной группы А-1 и тестовой группы В

Проверим гипотезу о том, что доли пользователей в группе А-1 и группе В одинаковы на каждом из шагов.

1. Нулевую гипотезу, которую проверяет тест, сформулируем так: "Между долями пользователей в группах нет статистически значимой разницы",
2. Альтернативную гипотезу, соответственно, сформулируем так: "Между долями пользователей в группах есть статистически значимая разница",
3. Примем уровень значимости в 5%.

Подготовим параметры для теста:

In [ ]:
sample_name = 'Группа А-1 и тестовая В'
params_of_test = {'sample_name': sample_name,
                  'alpha': 0.05,
                  'zero_hypothesis': 'Между долями пользователей нет различий',
                  'alt_hypothesis': 'Между долями пользователей есть различия',
                  'test': 'z_test'}

Проведем тест и выведем результат:

In [ ]:
tests.test_routine(groups_labels=[['A_one'], ['B_test']], sample_name=sample_name)

Выведем результат:

In [ ]:
report_styler(tests.result[tests.result['Выборка'].str.startswith('Группа А-1')])

&#9889; **Выводы**

В отношении различий долей между контрольной группой А-1 и тестовой группой В, исходя из представленных данных, на уровне значимости 5% - нет оснований отвергнуть нулевую гипотезу в пользу альтернативы: между долями пользователей в группе А-1 и тестовой группе В нет статистически значимой разницы.

[В оглавление](#TOC)

### - проведем статистические тесты в отношении контрольной группы А-2 и тестовой группы В

Проверим гипотезу о том, что доли пользователей в группе А-2 и группе В одинаковы на каждом из шагов.

1. Нулевую гипотезу, которую проверяет тест, сформулируем так: "Между долями пользователей в группах нет статистически значимой разницы",
2. Альтернативную гипотезу, соответственно, сформулируем так: "Между долями пользователей в группах есть статистически значимая разница",
3. Примем уровень значимости в 5%.

Подготовим параметры для теста:

In [ ]:
sample_name = 'Группа А-2 и тестовая В'
params_of_test = {'sample_name': sample_name,
                  'alpha': 0.05,
                  'zero_hypothesis': 'Между долями пользователей нет различий',
                  'alt_hypothesis': 'Между долями пользователей есть различия',
                  'test': 'z_test'}

Проведем тест:

In [ ]:
tests.test_routine(groups_labels=[['A_two'], ['B_test']], sample_name=sample_name)

Выведем результат:

In [ ]:
report_styler(tests.result[tests.result['Выборка'].str.startswith('Группа А-2')])

&#9889; **Выводы**

В отношении различий долей между контрольной группой А-2 и тестовой группой В, исходя из представленных данных, на уровне значимости 5% - нет оснований отвергнуть нулевую гипотезу в пользу альтернативы: между долями пользователей в группе А-2 и тестовой группе В нет статистически значимой разницы.

[В оглавление](#TOC)

### - проведем статистические тесты в отношении объединенных контрольных групп и тестовой группы В

Проверим гипотезу о том, что доли пользователей в контрольных группах и группе В одинаковы на каждом из шагов.

1. Нулевую гипотезу, которую проверяет тест, сформулируем так: "Между долями пользователей в группах нет статистически значимой разницы",
2. Альтернативную гипотезу, соответственно, сформулируем так: "Между долями пользователей в группах есть статистически значимая разница",
3. Примем уровень значимости в 5%.

Подготовим параметры для теста:

In [ ]:
sample_name = 'Объединенные А и тестовая В'
params_of_test = {'sample_name': sample_name,
                  'alpha': 0.05,
                  'zero_hypothesis': 'Между долями пользователей нет различий',
                  'alt_hypothesis': 'Между долями пользователей есть различия',
                  'test': 'z_test'}

In [ ]:
tests.test_routine(groups_labels=[['A_one', 'A_two'], ['B_test']],
                   sample_name=sample_name, united=True)

In [ ]:
report_styler(tests.result[tests.result['Выборка'].str.startswith('Объединенные')])

&#9889; **Выводы**

В отношении различий долей между объединненными контрольными группами и тестовой группой В, исходя из представленных данных, на уровне значимости 5% - нет оснований отвергнуть нулевую гипотезу в пользу альтернативы: между долями пользователей в контрольных группах и тестовой группе В нет статистически значимой разницы.

[В оглавление](#TOC)

### - комментарий по уровню значимости

Было проведено шестнадцать тестов на одних и тех же данных, что должно повышать требования к уровню значимости. В каждом тесте использован уровень значимости - 5%. Можно было бы использовать поправку Бонферрони:

In [ ]:
alpha = 0.05
bonferroni_alpha = alpha / 16
bonferroni_alpha

С поправкой уровень значимости должен быть установлен на уровне 0.3%. Проверим самое низкое p-value в результатах:

In [ ]:
tests.result['p-value'].min()

Принимая во внимание, что самое низкое p-value равно 0.07 поправка уровня значимости лишена практического смысла.

[В оглавление](#TOC)

# Выводы<a class='anchor' id='conclusions'></a>

1. В представленных данных 244126 наблюдений за две недели: от 25 июля 2019 года до 07.08.2019 года. Данные идут последовательно, без явных пропусков, без явных проблем с типом данных.
2. Выявлено, что за период от 25.07.2019 года до 31.07.2019 года зафиксировано чуть более 1% всех наблюдений о пользователях (2 826).
3. После исключения полных дубликатов (413 наблюдений) и неполных данных за период от 25.07.2019 года до 31.07.2019 года – получили таблицу с 240 887 строками и 6 колонками, без пропусков и проблем с данными. 
4. Дополнительно переименовали группы и события для улучшения восприятия. Группы 246 – А_one (A-1), 247 – A_two (A-2), 248 – B_test.
5. Всего в датасете представлено пять событий (с учетом переименования):
    - main screen (посетило 7 439 уникальных пользователей из 7 551, 98.47%)
    - offer screen (посетило 4613 уникальных пользователей из 7 551, 60.96%)
    - cart (посетило 3749 уникальных пользователей из 7 551, 49.56%)
    - payment (посетило 3547 уникальных пользователей из 7 551, 46.97%)
    - tutorial (посетило 847 уникальных пользователей из 7 551, 11.15%)
    - В среднем, на пользователя приходится 32.27 событий
7. Опираясь на число и долю пользователей и повременное появление событий в таблице с некоторой осторожностью, можно утверждать, что посетители интернет-магазина до оплаты товара проходят такие шаги:
    - Попадают на экран с обучением,
    - Заходят на главную страницу магазина.
    - Переходят на страницу товара.
    - Добавляют товар в корзину.
    - Переходят в корзину, оформляют заказ.
    - Оплачивают заказ.
8. Учитывая, что обучение проходят всего 11.15% уникальных пользователей и, разумно предположить, обучение не загружается каждый раз, этот шаг является необязательной частью взаимодействия с магазином. Как следствие шаг исключен при дальнейшем анализе.
9. Только 46.97% (3 539) пользователей прошли от первого до последнего этапа.
10. Из 7 419 уникальных пользователей 2 826 (37%) так и не перешли с главной страницы - это шаг с наибольшей потерей. Кроме того, при переходе с экрана предложений 859 (18%) из 4593 перешедших на экран предложений пользователей не перешли к оформлению заказа. Только 195 (4%) из 3734 так и не оплатили товар.
11. Установить причину потери пользователей на каждом из шагов с разумной степенью достоверности исходя из данных не представляется возможным.
12. Всего в таблице три группы - две контрольных, одна тестовая:
    - 2537 (33.67% от общего числа уникальных пользователей),
    - 2513 (33.36% от общего числа уникальных пользователей),
    - 2484 (32.97% от общего числа уникальных пользователей).
13. К данным был применен Z-тест для проверки гипотезы о равенстве долей в группах.
14. Разделение групп было произведено корректно: разница в количестве пользователей менее одного процента, статистически значимых различий между контрольными группами не выявлено.
15. По результатам статистического теста в отношении контрольных и тестовой группы, на уровне значимости 5%, не удалось опровергнуть гипотезу о равенстве долей пользователей на каждом шаге. Как следствие, следует сделать вывод о том, что изменение шрифта не повлияло на пользователей с точки зрения конверсии.


[В оглавление](#TOC)